In [29]:
import requests
from bs4 import BeautifulSoup
import bs4.element
import datetime
import warnings
warnings.filterwarnings('ignore')
import re
import pandas as pd
import time
from datetime import datetime

In [32]:
news_df = pd.DataFrame(
    index=None,
    columns={'date':news_dates,'title':news_titles,'link':news_urls,'content':news_contents}
)

In [ ]:
# ConnectionError방지
headers = {'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36'}

# BeautifulSoup 객체 생성
def get_soup_obj(url):
    res = requests.get(url, headers = headers, verify=False)
    soup = BeautifulSoup(res.text,'lxml')
    
    return soup

In [33]:
# 뉴스의 기본 정보 가져오기
def get_top3_news_info():
    news_urls =[]
    
    # 해당 분야 상위 뉴스 목록 주소
    sec_url = "https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1=101"
    
    # 해당 분야 상위 뉴스 HTML 가져오기
    soup = get_soup_obj(sec_url)
  
    # 해당 분야 상위 뉴스 3개 가져오기

    lis3 = soup.find('ul', class_='type06_headline').find_all("li", limit=3)
    for li in lis3:
        news_url = li.a.attrs.get('href')
        news_urls.append(news_url)
    
    print(news_urls)
    return news_urls

In [35]:
# 상위 3개 뉴스 크롤링
news_urls = get_top3_news_info()

for url in news_urls:
    news_html = get_soup_obj(url)


    # 뉴스 제목 가져오기
    title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    if title == None:
        title = news_html.select_one("#content > div.end_ct > div > h2")

    # 뉴스 본문 가져오기
    content = news_html.select("div#dic_area")
    if content == []:
        content = news_html.select("#articeBody")
        
    # 기사 텍스트만 가져오기
    # list합치기
    content = ''.join(str(content))

    # html태그제거 및 텍스트 다듬기
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1, repl='', string=str(title))
    content = re.sub(pattern=pattern1, repl='', string=content)
    pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    content = content.replace(pattern2, '')

    news_titles.append(title)
    news_contents.append(content)

    html_date = news_html.select_one("div#ct > div.media_end_head > div.media_end_head_info > div span")
    news_date = html_date.attrs['data-date-time']
    news_dates.append(news_date)

['https://n.news.naver.com/mnews/article/028/0002646939?sid=101', 'https://n.news.naver.com/mnews/article/031/0000756963?sid=101', 'https://n.news.naver.com/mnews/article/144/0000898251?sid=101']


In [36]:
news_df = pd.DataFrame({'date':news_dates,'title':news_titles,'link':news_urls,'content':news_contents})

#중복 행 지우기
news_df = news_df.drop_duplicates(keep='first',ignore_index=True)
print("중복 제거 후 행 개수: ",len(news_df))

중복 제거 후 행 개수:  3


In [37]:

def add_df() :
    a = pd.DataFrame({'date':news_dates,'title':news_titles,'link':news_urls,'content':news_contents})
    
    #중복 행 지우기
    a = a.drop_duplicates(keep='first',ignore_index=True)
    print("중복 제거 후 행 개수: ",len(a))

    pd.concat([news_df, a])

    # #중복 행 지우기
    # news_df = news_df.drop_duplicates(keep='first',ignore_index=True)
    # print("중복 제거 후 행 개수: ",len(news_df))

    print(news_df.head())

In [39]:
titles = []
contents =[]
dates = []

t=1
while t<5 :
    news_urls = get_top3_news_info()

    for url in news_urls:
        news_html = get_soup_obj(url)

        # 뉴스 제목 가져오기
        title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
        if title == None:
            title = news_html.select_one("#content > div.end_ct > div > h2")

        # 뉴스 본문 가져오기
        content = news_html.select("div#dic_area")
        if content == []:
            content = news_html.select("#articeBody")
            
        # 기사 텍스트만 가져오기
        # list합치기
        content = ''.join(str(content))

        # html태그제거 및 텍스트 다듬기
        pattern1 = '<[^>]*>'
        title = re.sub(pattern=pattern1, repl='', string=str(title))
        content = re.sub(pattern=pattern1, repl='', string=content)
        pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
        content = content.replace(pattern2, '')

        news_titles.append(title)
        news_contents.append(content)

        html_date = news_html.select_one("div#ct > div.media_end_head > div.media_end_head_info > div span")
        news_date = html_date.attrs['data-date-time']
        news_dates.append(news_date)
    
    add_df()

    time.sleep(30)

    t += 1

['https://n.news.naver.com/mnews/article/016/0002165895?sid=101', 'https://n.news.naver.com/mnews/article/015/0004864938?sid=101', 'https://n.news.naver.com/mnews/article/648/0000017710?sid=101']


ValueError: All arrays must be of the same length

In [ ]:
news_df.head()

,date,title,link,content
0,2023-07-06 00:16:01,박원순 시장 때 멈춘 서울 재건축 재개발 사업 재개,https://n.news.naver.com/mnews/article/021/000...,[\n\n\n\n\n서울시 달동네 모습. 게티이미지뱅크 오세훈 시장 핵심사업 ‘신...
1,2023-07-06 00:14:01,"파이퍼샌들러,""코인베이스 중립 하향""",https://n.news.naver.com/mnews/article/015/000...,"[\n암호자산 규제 방안 불확실하고 거래량 지속 줄어""2년래 최저 거래량과 사용자 ..."
2,2023-07-06 00:07:43,"남성현 산림청장 ""강원세계산림엑스포 적극 지원""",https://n.news.naver.com/mnews/article/654/000...,[\n춘천 방문서 성공 개최 협력 약속\n\n\n\n▲ 춘천목재산업단지 준공식이 5...


In [ ]:
###데이터 프레임으로 만들기###
from datetime import date, timedelta
import pandas as pd

#데이터 프레임 만들기

# list=[]
a = pd.DataFrame({'date':news_dates,'title':news_titles,'link':news_urls,'content':news_contents})

# list.append([news_df,a])

#데이터 프레임 저장
now = datetime.now() 
a.to_csv('C:\\big16\\trading_project\\ssuyan\\REAL_NEWS\\_{}.csv'.format(now.strftime('%Y%m%d_%H%M%S')),encoding='utf-8-sig',index=False)
